# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [4]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [5]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-03-15'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-EUR',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2625,2023-03-10,"$20,589.51","$20,665.95","$19,103.75","$19,235.70","$19,235.70",28683645998
2626,2023-03-11,"$19,239.46","$19,235.54","$18,531.49","$18,943.59","$18,943.59",37139999552
2627,2023-03-12,"$18,944.18","$19,511.58","$18,832.31","$19,379.53","$19,379.53",28347620356
2628,2023-03-13,"$19,375.41","$20,769.54","$19,207.07","$20,749.80","$20,749.80",27410915939
2629,2023-03-14,"$20,742.74","$22,875.22","$20,541.52","$22,565.94","$22,565.94",46130944785


In [56]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [36]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date','Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df= df.rename(columns=new_names)
df

,ds,y
0,2016-01-01,$396.76
1,2016-01-02,$400.28
2,2016-01-03,$399.32
3,2016-01-04,$396.41
4,2016-01-05,$400.17
...,...,...
2625,2023-03-10,"$20,589.51"
2626,2023-03-11,"$19,239.46"
2627,2023-03-12,"$18,944.18"
2628,2023-03-13,"$19,375.41"


In [37]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [39]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Evolución BTC-EUR"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [44]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [45]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [46]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2990,2024-03-09
2991,2024-03-10
2992,2024-03-11
2993,2024-03-12
2994,2024-03-13


In [47]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
2990,2024-03-09
2991,2024-03-10
2992,2024-03-11
2993,2024-03-12


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [49]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2016-01-01,$217.26,"$-3,065.98","$3,329.03",$217.26,$217.26,$0.08,$0.08,$0.08,$-0.00,$-0.00,$-0.00,$0.08,$0.08,$0.08,$0.00,$0.00,$0.00,$235.15
1,2016-01-02,$218.59,"$-3,086.90","$3,109.50",$218.59,$218.59,$0.09,$0.09,$0.09,$-0.00,$-0.00,$-0.00,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$237.46
2,2016-01-03,$219.92,"$-2,760.17","$3,388.87",$219.92,$219.92,$0.09,$0.09,$0.09,$-0.00,$-0.00,$-0.00,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$239.74
3,2016-01-04,$221.25,"$-2,656.55","$3,298.64",$221.25,$221.25,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$241.85
4,2016-01-05,$222.58,"$-2,704.41","$3,425.15",$222.58,$222.58,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,$243.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,2024-03-09,"$-8,810.52","$-26,364.66","$6,977.78","$-22,448.05","$6,238.72",$0.15,$0.15,$0.15,$-0.00,$-0.00,$-0.00,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,"$-10,133.31"
2991,2024-03-10,"$-8,869.91","$-26,476.07","$6,904.68","$-22,554.66","$6,259.22",$0.15,$0.15,$0.15,$-0.00,$-0.00,$-0.00,$0.15,$0.15,$0.15,$0.00,$0.00,$0.00,"$-10,226.47"
2992,2024-03-11,"$-8,929.31","$-26,978.36","$6,945.02","$-22,658.39","$6,277.24",$0.16,$0.16,$0.16,$0.00,$0.00,$0.00,$0.16,$0.16,$0.16,$0.00,$0.00,$0.00,"$-10,328.99"
2993,2024-03-12,"$-8,988.70","$-27,108.13","$6,237.78","$-22,748.63","$6,292.00",$0.16,$0.16,$0.16,$0.00,$0.00,$0.00,$0.16,$0.16,$0.16,$0.00,$0.00,$0.00,"$-10,433.85"


In [50]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2990,2024-03-09,"$-10,133.31","$-26,364.66","$6,977.78"
2991,2024-03-10,"$-10,226.47","$-26,476.07","$6,904.68"
2992,2024-03-11,"$-10,328.99","$-26,978.36","$6,945.02"
2993,2024-03-12,"$-10,433.85","$-27,108.13","$6,237.78"
2994,2024-03-13,"$-10,526.66","$-26,665.19","$6,764.02"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [51]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [52]:
plot_components_plotly(m, forecast)